In [77]:
#!pip install yfinance --upgrade
#!pip install talib-binary
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib

In [78]:
import os
import sys
import copy
import datetime
import time
import itertools

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from talib import MA_Type

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
#import own data manipulation toolkit
#sys.path.append('/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data')
#import _datatoolkit as dtk

**Defining Data Folder Structures**

In [81]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

**1) Define Tickers List, Backtest Date Range and Data Retrival Intervals**

In [82]:
# define list of tickers to backtest
#ticker_list = ['ARCC', 'STAG', 'O', 'WPC', 'MAIN', 'ABR', 'ACRE', 'DIVO', 'JEPI', 'LTC', 'OHI', 'PFLT', 'QYLD', 'SCHD', 'STOR', 'XRMI', 'TQQQ', 'TSLA', 'AAPL', 'GLD', 'SLV', 'GDX']
ticker_list = ['IRM', 'WPC', 'UVXY', 'SQQQ', 'MOS', 'ARCC', 'HTGC', 'ACRE', 'ABR', 'CTO', 'O', 'IBM', 'CAH', 'DLR', 'ORCC']
#ticker_list = ['AAPL', 'GOOG', 'AMZN', 'IBM', 'MU', 'GLD', 'SLV', 'QQQ', 'IWM', 'GME']

# define date range
start_date = '2021-03-20'
end_date = '2022-12-31'
interval = '1d'

# initial capital in USD
initial_capital = 8000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [83]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in ticker_list:
  try:
    globals()[ticker] = pd.DataFrame
    globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50)
    globals()[ticker]['100EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=100)
    globals()[ticker]['200EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=200) 
    globals()[ticker]['ATR'] = ta.ATR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['RSI'] = ta.RSI(globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)

    # Bollinger Bands
    globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # Stochastic indicators (original: fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3)
    globals()[ticker]['Stoch_slowk'], globals()[ticker]['Stoch_slowd'] = ta.STOCH(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'],\
                                                                         fastk_period=21, slowk_period=5, slowk_matype=0, slowd_period=5, slowd_matype=0)
    # Generating Ichimoku Cloud Indicators
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    globals()[ticker]['IC_per_26H'] = globals()[ticker]['High'].rolling(window=26).max()
    globals()[ticker]['IC_per_26L'] = globals()[ticker]['Low'].rolling(window=26).max()
    globals()[ticker]['IC_base'] = ((globals()[ticker]['IC_per_26H'] + globals()[ticker]['IC_per_26L'])/2)

    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    globals()[ticker]['IC_per_9H'] = globals()[ticker]['High'].rolling(window=9).max()
    globals()[ticker]['IC_per_9L'] = globals()[ticker]['Low'].rolling(window=9).max()
    globals()[ticker]['IC_conv'] = (globals()[ticker]['IC_per_9H'] + globals()[ticker]['IC_per_9L'])/2

    # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    globals()[ticker]['IC_span_a'] = ((globals()[ticker]['IC_base'] + globals()[ticker]['IC_conv'])/2).shift(26)
    
    # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    globals()[ticker]['IC_per_52H'] = globals()[ticker]['High'].rolling(window=52).max()
    globals()[ticker]['IC_per_52L'] = globals()[ticker]['Low'].rolling(window=52).max()
    globals()[ticker]['IC_span_b'] = ((globals()[ticker]['IC_per_52H'] + globals()[ticker]['IC_per_52L'])/2).shift(26)
    
    # add/drop columns and clean up
    globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
    globals()[ticker] = globals()[ticker].drop(['IC_per_26H', 'IC_per_26L', 'IC_per_9H', 'IC_per_9L', 'IC_per_52H', 'IC_per_52L'], axis=1)
    globals()[ticker] = globals()[ticker].round(4).dropna()
    globals()[ticker] = globals()[ticker].reset_index()
    globals()[ticker]['Position'] = 0
    globals()[ticker][['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

**Other Optional Indicators:** </b>

**MACD_hist** - globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9) </b>

**ADX** - globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>


**ADXR** - globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>

**50 Day Exponential Moving Average** - globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50) </b>

**Bollinger Bands** - globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3


**3) Building the DataFrame List, Perform Backtesting**

In [84]:
#df_list = [ARCC, STAG, O, WPC, MAIN, ABR, ACRE, DIVO, JEPI, LTC, OHI, PFLT, QYLD, SCHD, STOR, XRMI, TQQQ, TSLA, AAPL, GLD, SLV, GDX]
df_list = [IRM, WPC, UVXY, SQQQ, MOS, ARCC, HTGC, ACRE, ABR, CTO, O, IBM, CAH, DLR, ORCC]
#df_list = [AAPL, GOOG, AMZN, IBM, MU, GLD, SLV, QQQ, IWM, GME]

**Back Testing** </b>


*   Backtesting of Trade Logic across all tickers and in the designated timeframe
*   Backtesting for Trade strategy with simulated PNL



In [85]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [86]:
# defining the global list for storing output
global macdrsi_trade_list
macdrsi_trade_list = []

global macdema_trade_list
macdema_trade_list = []

global ic_trade_list
ic_trade_list = []

global bbrsi_trade_list
bbrsi_trade_list = []

pos_size = round(initial_capital/len(ticker_list),2)

3.1) Trade Strategy - MACD + RSI



*   Entry - MACD crosses over signal line + RSI < 45
*   Exit - MACD crosses below signal line or RSI > 70



In [87]:
def backtest_macdrsi(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.85
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_rsi         = df.loc[i, 'RSI']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_MACD > now_MACD_signal) and (now_rsi < 45)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      macdrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and (now_MACD < now_MACD_signal)) or \
      ((pos_opened == True) and (now_rsi > 70)) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])) and (now_open*lot_size < stop_loss):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      macdrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

3.2) Trade Strategy - MACD EMA </br>


*   Entry - MACD > signal + price > 200 EMA
*   Exit - MACD < signal + price < 200 EMA
*   Risk Rward Ratio at 1 : 1.5






In [88]:
def backtest_macdema(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.8
  profit_target = initial_equity_value * 1.3

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_100EMA      = df.loc[i, '100EMA']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi         = df.loc[i, 'RSI']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_MACD > now_MACD_signal) and (now_open > now_200EMA) and (now_50EMA > now_200EMA)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      macdema_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_MACD < now_MACD_signal) or (now_open < now_200EMA))) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
#      ((pos_opened == True) and (now_open*lot_size < stop_loss)) or \
#      ((pos_opened == True) and (now_open*lot_size > profit_target)):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      macdema_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

3.3) Trade Strategy - Stochastic + EMA </br>


*   Entry - STOCH K > 80 + MACD cross-over 
*   Exit - STOCH K < 80 + MACD < signal 





In [89]:
def backtest_stochema(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_50EMA      = df.loc[i, '50EMA']
    now_100EMA      = df.loc[i, '100EMA']
    now_200EMA      = df.loc[i, '200EMA']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_Stoch_slowk = df.loc[i, 'Stoch_slowk']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open > now_100EMA) and (now_Stoch_slowk <30)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      bbrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open < now_200EMA) or (now_Stoch_slowk > 65))) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bbrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

3.4) Trade Strategy - Bollinger Bands + MACD </br>


*   Entry - Price > Bollinger Bands Mid line + MACD > signal line 
*   Exit - Price > Bollinger Bands Upper line or MACD < signal line





In [90]:
def backtest_bbmacd(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_100EMA      = df.loc[i, '100EMA']
    now_200EMA      = df.loc[i, '200EMA']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open > now_bband_m) and (now_MACD > now_MACD_signal)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      bbrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open > now_bband_u) or (now_MACD < now_MACD_signal))) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bbrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

3.5) Trade Strategy - Bollinger Bands + ADX + RSI </br>


*   Entry - Price < Bollinger Bands Lower line + ADX > 25 & RSI < 30 
*   Exit - Price > Bollinger Bands Upper line + ADX > 25 & RSI > 70





In [91]:
def backtest_bbadxrsi(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_adx         = df.loc[i, 'ADX']
    now_rsi         = df.loc[i, 'RSI']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open < now_bband_l) and (now_adx > 20) and (now_rsi < 35)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      bbrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open > now_bband_u) or now_rsi > 65)) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bbrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

3.4) Trade Strategy - Ichimoku Cloud </br>


*   Entry - price > 200 EMA and & span a + conversion line > baseline 
*   Exit - price < 200 EMA or span b + conversion line < baseline
*   Stop loss > baseline






In [92]:
def backtest_ic(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.8
  profit_target = initial_equity_value * 1.3

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_ic_conv     = df.loc[i, 'IC_conv']
    now_ic_base     = df.loc[i, 'IC_base']
    now_ic_span_a   = df.loc[i, 'IC_span_a']
    now_ic_span_b   = df.loc[i, 'IC_span_b']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_ic_conv > now_ic_base) and (now_open > now_ic_span_a)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      ic_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True) and ((now_ic_conv < now_ic_base) or (now_open < now_ic_span_b)):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      ic_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

**4) Projecting Total PNL** </br>


*   Creating for loop to simulate PNL based on particular strategy
*   Calculate portfolio PNL, ROI etc.



In [93]:
bbadxrsi_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbadxrsi(df)
    bbadxrsi_result_list.append(test_result)

    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbadxrsi_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  

In [114]:
# PNL for Bollinger Bands + ADX + RSI Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + ADX + RSI')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbadxrsi_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbadxrsi_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbadxrsi_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbadxrsi_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: Bollinger Bands + ADX + RSI
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 10
Total PNL: USD 537.38
Required Capital: USD 8000
ROI:  6.72 %
Avg Win Rate:  87.5 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
3,SQQQ,1,180.43,180.43,199.20,17.00,100.0
4,MOS,1,146.05,146.05,158.00,30.00,100.0
13,DLR,3,107.31,35.77,77.30,11.33,100.0
10,O,1,75.51,75.51,81.44,20.00,100.0
1,WPC,1,71.32,71.32,70.42,20.00,100.0
8,ABR,1,29.13,29.13,26.10,3.00,100.0
11,IBM,1,28.45,28.45,25.57,9.00,100.0
2,UVXY,1,-100.82,-100.82,46.47,24.00,0.0


In [95]:
bbmacd_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbmacd(df)
    bbmacd_result_list.append(test_result)

    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbmacd_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [115]:
# PNL for Bollinger Bands + MACD Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + MACD')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbmacd_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbmacd_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbmacd_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbmacd_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)


Trading Strategy: Bollinger Bands + MACD
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 165
Total PNL: USD 3272.17
Required Capital: USD 8000
ROI:  40.9 %
Avg Win Rate:  58.75 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
3,SQQQ,12,830.93,69.24,322.56,9.85,75.00
4,MOS,11,334.63,30.42,277.29,12.00,72.73
2,UVXY,10,251.21,25.12,265.49,10.20,50.00
12,CAH,9,231.93,25.77,176.88,12.33,55.56
0,IRM,9,226.56,25.17,100.43,13.67,44.44
8,ABR,14,174.44,12.46,114.37,7.27,50.00
10,O,6,165.98,27.66,74.94,20.29,50.00
7,ACRE,11,162.59,14.78,83.30,11.18,36.36
6,HTGC,12,160.33,13.36,81.56,8.92,66.67
14,ORCC,10,143.41,14.34,117.32,12.20,80.00


In [105]:
macdema_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_macdema(df)
    macdema_result_list.append(test_result)

    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = macdema_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [113]:
# PNL for MACD EMA Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: MACD + 200 EMA')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((macdema_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = macdema_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (macdema_result_df['No of Trade'].sum()))
print('Total PNL: USD', macdema_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(macdema_result_df[macdema_result_df['No of Trade'] > 0]['Win Rate %'].mean(), 2), '%')
macdema_result_df[macdema_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: MACD + 200 EMA
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 0
Total PNL: USD 0
Required Capital: USD 8000
ROI:  0.0 %
Avg Win Rate:  nan %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %


In [116]:
stochema_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_stochema(df)
    stochema_result_list.append(test_result)

    stochema_result_df = pd.DataFrame(stochema_result_list)
    stochema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = stochema_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    stochema_result_df = pd.DataFrame(stochema_result_list)
    stochema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [117]:
# PNL for Stochasti Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Stochastic + EMA')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((stochema_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = stochema_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (stochema_result_df['No of Trade'].sum()))
print('Total PNL: USD', stochema_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(stochema_result_df[stochema_result_df['No of Trade'] > 0]['Win Rate %'].mean(), 2), '%')
stochema_result_df[stochema_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: Stochastic + EMA
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 29
Total PNL: USD -393.91
Required Capital: USD 8000
ROI:  -4.92 %
Avg Win Rate:  24.36 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
2,UVXY,1,30.07,30.07,331.66,9.82,100.00
9,CTO,3,9.34,3.11,63.51,8.65,66.67
3,SQQQ,4,0.50,0.12,322.56,10.17,25.00
12,CAH,2,0.47,0.24,176.88,13.50,50.00
11,IBM,1,-9.12,-9.12,114.98,8.33,0.00
6,HTGC,2,-22.58,-11.29,153.35,9.69,0.00
14,ORCC,2,-24.22,-12.11,117.32,12.50,50.00
5,ARCC,2,-24.32,-12.16,79.40,9.83,0.00
1,WPC,4,-32.93,-8.23,79.59,11.00,25.00
7,ACRE,1,-43.49,-43.49,83.30,13.58,0.00


In [118]:
macdrsi_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    macdrsi_test_result = backtest_macdrsi(df)
    macdrsi_result_list.append(macdrsi_test_result)

    macdrsi_result_df = pd.DataFrame(macdrsi_result_list)
    macdrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = macdrsi_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    macdrsi_result_df = pd.DataFrame(macdrsi_result_list)
    macdrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [119]:
# PNL for MACD RSI Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: MACD + RSI')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((macdrsi_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = macdrsi_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (macdrsi_result_df['No of Trade'].sum()))
print('Total PNL: USD', macdrsi_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(macdrsi_result_df[macdrsi_result_df['No of Trade'] > 0]['Win Rate %'].mean(), 2), '%')
macdrsi_result_df[macdrsi_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: MACD + RSI
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 47
Total PNL: USD 1719.88
Required Capital: USD 8000
ROI:  21.5 %
Avg Win Rate:  67.11 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
3,SQQQ,1,302.83,302.83,322.56,10.17,100.00
8,ABR,5,186.73,37.35,129.28,8.83,40.00
2,UVXY,3,179.51,59.84,331.66,9.82,66.67
6,HTGC,4,161.70,40.42,153.35,9.69,50.00
1,WPC,4,120.90,30.22,79.59,11.00,75.00
4,MOS,3,117.72,39.24,277.29,14.60,66.67
13,DLR,4,108.65,27.16,97.75,11.25,50.00
5,ARCC,3,98.74,32.91,79.40,9.83,66.67
14,ORCC,3,83.27,27.76,117.32,12.50,66.67
0,IRM,4,80.90,20.23,100.43,12.67,50.00


In [120]:
ic_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    ic_test_result = backtest_ic(df)
    ic_result_list.append(ic_test_result)

    ic_result_df = pd.DataFrame(ic_result_list)
    ic_result_list.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = ic_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    ic_result_df = pd.DataFrame(ic_result_list)
    ic_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [121]:
# PNL for Ichimoku Cloud Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Ichimoku Cloud Strategy')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((ic_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = ic_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (ic_result_df['No of Trade'].sum()))
print('Total PNL: USD', ic_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(ic_result_df[ic_result_df['No of Trade'] > 0]['Win Rate %'].mean(), 2), '%')
ic_result_df[ic_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: Ichimoku Cloud Strategy
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 0
Total PNL: USD 0
Required Capital: USD 8000
ROI:  0.0 %
Avg Win Rate:  nan %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %


**5) Detailed Portfolio Transaction List**

In [104]:
# Class to call Pandas by row and column
# https://stackoverflow.com/questions/28754603/indexing-pandas-data-frames-integer-rows-named-columns 
class XLocIndexer:
    def __init__(self, frame):
        self.frame = frame
    
    def __getitem__(self, key):
        row, col = key
        return self.frame.iloc[row][col]

pd.core.indexing.IndexingMixin.xloc = property(lambda frame: XLocIndexer(frame))

Appendix - Ichimoku Cloud </br>
https://stackoverflow.com/questions/28477222/python-pandas-calculate-ichimoku-chart-components </br>

1) Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2)) </br>
- period9_high = pd.rolling_max(high_prices, window=9) </br>
- period9_low = pd.rolling_min(low_prices, window=9) </br>
- tenkan_sen = (period9_high + period9_low) / 2 </br>

2) Kijun-sen (Base Line): (26-period high + 26-period low)/2)) </br>
- period26_high = pd.rolling_max(high_prices, window=26) </br>
- period26_low = pd.rolling_min(low_prices, window=26) </br>
- kijun_sen = (period26_high + period26_low) / 2 </br>

3) Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2)) </br>
- senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26) </br>

4) Senkou Span B (Leading Span B): (52-period high + 52-period low)/2)) </br>
- period52_high = pd.rolling_max(high_prices, window=52) </br>
- period52_low = pd.rolling_min(low_prices, window=52) </br>
- senkou_span_b = ((period52_high + period52_low) / 2).shift(26) </br>
